This notebook essentially re-implements some of the examples from Think Stats 2e, by Allen Downey. I've tried to keep the names and the structure as similar as is reasonable, so that these examples can be clearly matched up with the Python examples from the book.

A major change is that I don't use an object-oriented approach. From a pedagogical viewpoint, I think this is unfortunate. The abstraction that Allen implements is highly informative. I think it does a better job demonstrating the core elements of hypothesis testing than any other approach I've seen.

However, in my limited experience, most people who work in R do not naturally develop in an object-oriented way<sup>1</sup>. So for my first pass here, I went with what I think is most accessible example rather than most elegant. When I'm a better R developer, perhaps I'll change my mind and rewrite it all.

<sup>1</sup> *Please correct me on this, if you have reason to believe differently.*

## test the fairness of a coin

If we want to evaluate whether a coin is fair or not, then we can simply compare the number of heads and number of tails that we get from our coin and compare it to the values that we would get from a fair coing. This can be done directly using the binomial distribution, but here we'll actually calculate the p-value using simulation.

In [ ]:
# the number of heads and tails
h <- 140
t <- 110

In [ ]:
# define the test statistic, in this case the difference between the number of heads and tails
coin_test.statistic  <- function(data) {
    heads <- data[1]
    tails <- data[2]
    test_stat  <- abs(heads - tails)
}

# and the model for the null hypothesis, 
# which in this case is simply to "flip" a fair coin
coin_test.run_model <- function(data) {
    n <- data[1] + data[2]
    x <- sample(2, n, replace = TRUE)
    result <- c(sum(x == 1), sum(x == 2))
    return(result)
}

In [ ]:
# flip the coin n times and see how often the result was
# more extreme than our actual data
iter  <- 1000

actual.data  <- c(h, t)
actual.statistic <- coin_test.statistic(actual.data)

sim.statistic  <- numeric(iter)
for (ii in 1:iter) {
    run.data <- coin_test.run_model(actual.data)
    sim.statistic[ii] <- coin_test.statistic(run.data)
}

In [ ]:
p  <- sum(sim.statistic > actual.statistic) / length(sim.statistic)

writeLines(sprintf("the p-value is %.2e", p))

## test a difference in means

Rather than the National Survey of Family Growth data (NSFG), I'll use a dataset on wine quality from Kaggle. You can download it from [here](https://www.kaggle.com/zynicide/wine-reviews). The methodology remains the same as in Think Stats, however.

In [ ]:
library(readr)
tb.red  <- read_delim("data//winequality-red.csv", delim = ";")
tb.white <- read_delim("data//winequality-white.csv", delim = ";")

In [ ]:
diff_means_permute.statistic <- function(data) {
    group.1 <- data[[1]]
    group.2 <- data[[2]]
    
    test.statistic  <- abs(mean(group.1) - mean(group.2))
    return(test.statistic)
}

diff_means_permute.run_model <- function(data) {
    group.1 <- data[[1]]
    group.2 <- data[[2]]
    
    pool <- c(group.1, group.2)
    
    n1  <- length(group.1)
    x  <- sample(pool)
    
    return(list(x[1:n1], x[(n1+1):length(x)]))
}

diff_means_permute.pvalue <- function(actual.data, iter = 1000) {
    actual.statistic <- diff_means_permute.statistic(actual.data)

    sim.statistic  <- numeric(iter)
    for (ii in 1:iter) {
        run.data <- diff_means_permute.run_model(actual.data)
        sim.statistic[ii] <- diff_means_permute.statistic(run.data)
    }

    p  <- sum(sim.statistic > actual.statistic) / length(sim.statistic)
    return(p)
}

In [ ]:
# permute the means to create random groups,
# compare simulated means to actual to get p-value

pvalue <- diff_means_permute.pvalue(list(tb.red$alcohol, tb.white$alcohol), iter = 1000)

writeLines(sprintf("the p-value is %.3e", pvalue))